In [85]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download("vader_lexicon")
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Angga\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Angga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

LABELLING USING LEXICON BASED APPROACH

In [86]:

# Baca data dari file Excel dan gabungkan
dataAnis = pd.read_excel('dataset/anies.xlsx')
dataPrabowo = pd.read_excel('dataset/prabowo.xlsx')
dataGanjar = pd.read_excel('dataset/ganjar.xlsx')
data = pd.concat([dataAnis, dataPrabowo, dataGanjar])

data = data[['full_text']]

sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["full_text"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["full_text"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["full_text"]]
data["Compound"] = [sentiments.polarity_scores(i)["compound"] for i in data["full_text"]]

score= data["Compound"].values
sentiment = []
for i in score:
  if i >= 0.05:
    sentiment.append('positif')
  elif i <= -0.05 :
    sentiment.append('negatif')
  else:
    sentiment.append('netral')
data["sentiment"] = sentiment

data = data[['full_text', 'sentiment']]

data['sentiment'].value_counts()

sentiment
netral     1626
positif      86
negatif      42
Name: count, dtype: int64

In [88]:
data['casefolding'] = data['full_text'].str.lower()

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus @..
    text = re.sub(r'#', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]+', '', text) # menghapus RT
    text = re.sub(r'https?:\/\/\S+', '', text) # menghapus url
    text = re.sub(r':\)', '', text) # menghapus icon
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text) # menghapus single character
    text = re.sub(r'\s+', ' ', text) # menghapus spasi ganda
    text = re.sub(r'[^\w\s]', '', text) # menghapus punctuation
    text = re.sub(r'\.+', '', text) #menghilangkan teks yang mengandung akhiran MULTIPLE TITIK (...)
    text = re.sub(r'\d+', '', text) #menghilangkan teks yang mengandung angka
    text = re.sub(r'\b\w{1,2}\b', '', text) #menghilangkan teks yang mengandung 1-2 huru
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.encode('ascii', 'ignore').decode('ascii') # menghapus non-ascii (EMOTICON)
    return text

data['cleansing'] = data['casefolding'].apply(clean_text)
# delete row with null value
data = data.dropna()
# menghapus row yang memiliki nilai text kosong
data = data[data['cleansing'] != '']
data.reset_index(drop=True, inplace = True)
# Menghapus text duplikat
data.cleansing.duplicated().sum()
data.loc[data.cleansing.duplicated(), :]
data.drop_duplicates(subset=['cleansing'], inplace=True)
data.reset_index(drop=True, inplace = True)
                
#TOKENIZING
def tokenizing(text):
    nstr = text.split(' ')
    md= []
    a = -1
    for hu in nstr:
        a = a + 1
    if hu == '':
        md.append(a)
    p = 0
    b = 0
    for q in md:
        b = q - p
        del nstr[b]
        p = p + 1
    return nstr
    
data['tokenizing'] = data['cleansing'].apply(tokenizing)

def stopword_removal(text):
    factory = stopwords.words('indonesian', 'english')
    x = []
    dkk = []
    def myFunc(x):
        if x in factory:
            return False
        else:
            return True
    fit = filter(myFunc, text)
    for x in fit:
        dkk.append(x)
    return dkk               
data['stopword_removal'] = data['tokenizing'].apply(stopword_removal)

from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Fungsi untuk stemming
def stemming(text):
    # Inisialisasi stemmer untuk bahasa Indonesia
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    # Stem setiap kata dalam teks
    stemmed_text = [stemmer.stem(word) for word in text]
    
    return stemmed_text

  
data['stemming']= data['stopword_removal'].apply(stemming)
data.head()

KeyboardInterrupt: 

In [82]:
data = data[['stemming', 'sentiment']]
data.rename(columns={'stemming':'text'}, inplace=True)
data.to_csv('dataset/data.csv', index=False)

TRAINING MULTINOMIAL NAIVE BAYES CLASSIFIER

In [83]:
# Baca data dari file CSV
df = pd.read_csv('dataset/data.csv')
df = df[['text', 'sentiment']]

# Bagi data menjadi data latih dan data uji
X = df['text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Ekstraksi fitur menggunakan CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Melatih model dengan Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Melakukan prediksi
y_pred = clf.predict(X_test)

# Menampilkan hasil
print("Accuracy:", accuracy_score(y_test, y_pred))

# Menampilkan classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 0.9197530864197531
Classification Report:
              precision    recall  f1-score   support

     negatif       0.33      0.11      0.17         9
      netral       0.93      0.99      0.96       299
     positif       0.50      0.06      0.11        16

    accuracy                           0.92       324
   macro avg       0.59      0.39      0.41       324
weighted avg       0.89      0.92      0.89       324

